# Unsupervised Learning as Signals for Pairs Trading and StatArb

We want to detect pairs in a universe of trading securities, using various unsupervised learning models. 

Our signal to trade is a stock that deviates substantially from its group of similar behaviour.

Data: https://www.kaggle.com/datasets/addarm/lagged-firm-characteristics-han2022.\
This dataset has been curated for the papers “Empirical Asset Pricing via Machine Learning”(2018) and “Autoencoder Asset Pricing Models.” (2019) by Shihao Gu, Bryan Kelly and Dacheng Xiu.

The dataset has 94 one-month-Lagged Firm Characteristics as the CRSP (Center for Research in Security Prices) releases these with a month delay.

The exercise is In-Sample.

In [1]:
# import libraries
import os
# 
import numpy as np
import pandas as pd
# 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from tqdm import tqdm
from utils import (
                get_log_rets,
                get_pnl,
                get_trade_stats,
                interpolate_with_median,
                plot_cluster_distributions,
                plot_cummulative_rets,
                plot_log_rets,
                process_trade_opportunities,
                train_agg_clusters,
                train_db_clusters,
                train_km_clusters,
                )

In [2]:
# globals
DATA_DIR = "data"
FILEPATH = f"archive/datashare.csv"

MIN_YEAR = 1980
MAX_YEAR = 2021
CHUNKS = 10000

FILEPATH_PARQ = f"{DATA_DIR}/historic_characteristics_{MIN_YEAR}_{MAX_YEAR}.parquet"
FILEPATH_MOM_PARQ = f"{DATA_DIR}/data_mom_{MIN_YEAR}_{MAX_YEAR}.parquet"
FILEPATH_CLEAN_PARQ = f"{DATA_DIR}/data_cleaning_{MIN_YEAR}_{MAX_YEAR}.parquet"
FILEPATH_PRE_PARQ = f"{DATA_DIR}/data_preprocessed_{MIN_YEAR}_{MAX_YEAR}.parquet"

AGG_CLUSTERS_FILEPATH = f"{DATA_DIR}/agg_clusters_data_{MIN_YEAR}_{MAX_YEAR}.pkl"
AGG_CLUSTERS_MEM_FILEPATH = f"{DATA_DIR}/agg_clusters_memdata_{MIN_YEAR}_{MAX_YEAR}.pkl"
KM_CLUSTERS_FILEPATH = f"{DATA_DIR}/km_clusters_data_{MIN_YEAR}_{MAX_YEAR}.pkl"
KM_CLUSTERS_MEM_FILEPATH = f"{DATA_DIR}/km_clusters_memdata_{MIN_YEAR}_{MAX_YEAR}.pkl"
KM_CLUSTERS_OPT_FILEPATH = f"{DATA_DIR}/km_clusters_data_opt_{MIN_YEAR}_{MAX_YEAR}.pkl"
KM_CLUSTERS_OPT_MEM_FILEPATH = f"{DATA_DIR}/km_clusters_memdata_opt_{MIN_YEAR}_{MAX_YEAR}.pkl"
KM_CLUSTERS_DTW_FILEPATH = f"{DATA_DIR}/km_clusters_data_dtw_{MIN_YEAR}_{MAX_YEAR}.pkl"
KM_CLUSTERS_DTW_MEM_FILEPATH = f"{DATA_DIR}/km_clusters_memdata_dtw_{MIN_YEAR}_{MAX_YEAR}.pkl"
KM_CLUSTERS_OPTPCA_FILEPATH = f"{DATA_DIR}/km_clusters_data_optpca_{MIN_YEAR}_{MAX_YEAR}.pkl"
KM_CLUSTERS_OPTPCA_MEM_FILEPATH = f"{DATA_DIR}/km_clusters_memdata_optpca_{MIN_YEAR}_{MAX_YEAR}.pkl"
DBS_CLUSTERS_FILEPATH = f"{DATA_DIR}/dbs_clusters_data_{MIN_YEAR}_{MAX_YEAR}.pkl"
DBS_CLUSTERS_MEM_FILEPATH = f"{DATA_DIR}/dbs_clusters_memdata_{MIN_YEAR}_{MAX_YEAR}.pkl"

SIGNALS_KM_FILEPATH = f"{DATA_DIR}/signals_km_{MIN_YEAR}_{MAX_YEAR}.pkl"
SIGNALS_KM_OPT_FILEPATH = f"{DATA_DIR}/signals_km_opt_{MIN_YEAR}_{MAX_YEAR}.pkl"
SIGNALS_KM_OPTPCA_FILEPATH = f"{DATA_DIR}/signals_km_optpca_{MIN_YEAR}_{MAX_YEAR}.pkl"
SIGNALS_KM_DTW_FILEPATH = f"{DATA_DIR}/signals_km_optdtw_{MIN_YEAR}_{MAX_YEAR}.pkl"
SIGNALS_DB_FILEPATH = f"{DATA_DIR}/signals_db_{MIN_YEAR}_{MAX_YEAR}.pkl"
SIGNALS_AGG_FILEPATH = f"{DATA_DIR}/signals_agg_{MIN_YEAR}_{MAX_YEAR}.pkl"
PCA_FILEPATH = f"{DATA_DIR}/pca_{MIN_YEAR}_{MAX_YEAR}.pkl"

FEATURES = [
    "DATE", "absacc", "acc", "aeavol", "age", "agr", "baspread", "beta", "betasq", "bm",
    "bm_ia", "cash", "cashdebt", "cashpr", "cfp", "cfp_ia", "chatoia", "chcsho", "chempia",
    "chinv", "chmom", "chpmia", "chtx", "cinvest", "convind", "currat", "depr", "divi",
    "divo", "dolvol", "dy", "ear", "egr", "ep", "gma", "herf", "hire", "idiovol", "ill",
    "indmom", "invest", "lev", "lgr", "maxret", "mom1m", "ms", "mve_ia", "mvel1", "nincr",
    "operprof", "pchcapx_ia", "pchcurrat", "pchdepr", "pchgm_pchsale", "pchquick",
    "pchsale_pchrect", "pctacc", "permno", "pricedelay", "ps", "quick", "rd", "retvol",
    "roaq", "roeq", "roic", "rsup", "salecash", "salerec", "securedind", "sgr", "sic2",
    "sin", "sp", "std_dolvol", "std_turn", "tang", "tb", "turn", "zerotrade"
]
WINDOW = 48
NAN_THRESHOLD = 0.15
MAX_VARIANCE = 0.99
RETS = 'rets'
OVER_RETS = 'over_rets'
UNDER_RETS = 'under_rets'
MOM_FEATURES = [f"mom{i}m" for i in range(1, WINDOW + 1)]

In [3]:
# read data
if not os.path.exists(FILEPATH_PARQ):
    chars_df = pd.read_csv(FILEPATH)[FEATURES]
    chars_df['DATE'] = pd.to_datetime(chars_df['DATE'], format='%Y%m%d')
    chars_df = chars_df[(chars_df['DATE'].dt.year >= MIN_YEAR) & (chars_df['DATE'].dt.year <= MAX_YEAR)]
    chars_df = chars_df.sort_values("DATE")
    chars_df.to_parquet(FILEPATH_PARQ, index=False, compression="snappy")
else:
    chars_df = pd.read_parquet(FILEPATH_PARQ)

chars_df.head(5)

,DATE,absacc,acc,aeavol,age,agr,baspread,beta,betasq,bm,...,sgr,sic2,sin,sp,std_dolvol,std_turn,tang,tb,turn,zerotrade
0,1980-01-31,0.113203,0.113203,1.001090,17.0,-0.173800,0.014234,1.060420,1.124491,1.724617,...,0.156975,37.0,0.0,3.063197,0.881844,0.635898,0.545459,0.173391,0.319863,1.115306e-07
1,1980-01-31,0.049921,0.049921,NaN,8.0,-0.266184,0.099243,1.378908,1.901387,2.326257,...,0.099281,59.0,0.0,8.509354,NaN,NaN,0.563272,0.204887,NaN,NaN
2,1980-01-31,0.038596,0.038596,2.862434,8.0,-0.041359,0.012620,1.147547,1.316864,2.244966,...,0.026621,39.0,0.0,4.129895,0.772788,0.600652,0.612665,-0.884532,0.214943,1.310880e-07
3,1980-01-31,0.148813,0.148813,0.208791,8.0,-0.077528,0.027943,0.536016,0.287313,2.053509,...,0.006643,51.0,0.0,7.211359,0.822114,0.520015,0.672020,0.309224,0.094820,7.350000e+00
4,1980-01-31,NaN,NaN,NaN,NaN,NaN,0.149658,NaN,NaN,NaN,...,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Firm Characteristics Data

All papers referenced in this article leverage the Center for Research in Security Prices (CRSP) research data on securities. Specifically they look at the company characteristics, presented in the table below:

| Acronym  | Firm characteristic                                           | Acronym    | Firm characteristic                                       |
|----------|--------------------------------------------------------------|------------|----------------------------------------------------------|
| absacc   | Absolute accruals                                            | invest     | Capital expenditures and inventory                        |
| acc      | Working capital accruals                                     | IPO        | New equity issue                                          |
| aeavol   | Abnormal earnings announcement volume                        | lev        | Leverage                                                  |
| age      | # years since first Compustat coverage                       | lgr        | Growth in long-term debt                                  |
| agr      | Asset growth                                                  | maxret     | Maximum daily return                                      |
| baspread | Bid-ask spread                                               | ms         | Financial statement score                                 |
| beta     | Beta                                                         | mve        | Size                                                      |
| betasq   | Beta squared                                                 | mve ia     | Industry-adjusted size                                    |
| bm       | Book-to-market                                               | nincr      | Number of earnings increases                              |
| bm ia    | Industry-adjusted book to market                             | operprof   | Operating profitability                                   |
| cash     | Cash holdings                                                | pchcapx ia | Industry adjusted % change in capital expenditures        |
| cashdebt | Cash flow to debt                                            | pchcurrat  | % change in current ratio                                 |
| cashpr   | Cash productivity                                            | pchdepr    | % change in depreciation                                  |
| cfp      | Cash flow to price ratio                                     | pchgm      | % change in gross margin                                  |
| cfp ia   | Industry-adjusted cash flow to price ratio                   | pchsale    | % change in sales                                         |
| chatoia  | Industry-adjusted change in asset turnover                   | pchquick   | % change in quick ratio                                   |
| chcsho   | Change in shares outstanding                                 | pctacc     | Percent accruals                                          |
| chempia  | Industry-adjusted change in employees                        | pricedelay | Price delay                                               |
| chinv    | Change in inventory                                          | ps         | Financial statements score                                |
| chmom    | Change in 6-month momentum                                   | quick      | Quick ratio                                               |
| chpmia   | Industry-adjusted change in profit margin                    | rd         | R&D increase                                              |
| chtx     | Change in tax expense                                        | retvol     | Return volatility                                         |
| cinvest  | Corporate investment                                         | roaq       | Return on assets                                          |
| convind  | Convertible debt indicator                                   | roeq       | Return on equity                                          |
| currat   | Current ratio                                                | roic       | Return on invested capital                                |
| depr     | Depreciation / PP&E                                          | rsup       | Revenue surprise                                          |
| divi     | Dividend initiation                                          | sgr        | Sales growth                                              |
| divo     | Dividend omission                                            | sin        | Sin stocks                                                |
| dolvol   | Dollar trading volume                                        | SP         | Sales to price                                            |
| dy       | Dividend to price                                            | std dolvol | Volatility of liquidity (dollar trading volume)          |
| ear      | Earnings announcement return                                 | std turn   | Volatility of liquidity (share turnover)                  |
| egr      | Growth in common shareholder equity                          | sue        | Unexpected quarterly earnings                             |
| ep       | Earnings to price                                            | tang       | Debt capacity/firm tangibility                            |
| gma      | Gross profitability                                          | tb         | Tax income to book income                                 |
| herf     | Industry sales concentration                                 | turn       | Share turnover                                            |
| hire     | Employee growth rate                                         | zerotrade  | Zero trading days                                         |
| idiovol  | Idiosyncratic return volatility                              |            |                                                           |
| ill      | Illiquidity                                                  |            |                                                           |
| indmom   | Industry momentum                                            |            |                                                           |



## Data Cleaning and Feature Engineering

To sanitize our dataset, we drop any company with insufficient data to fill a window of 48 months and drop characteristics with more than 15% nans. We then impute any missing characteristic with the median of the current time window.

The data has a one month momentum (MOM1M) characteristic. MOM is the stock’s acceleration measure, and is calculated as follows:

$$
\text{MOM}_{\text{1 month}} = \left( \frac{\text{Price at End of Month} - \text{Price at Start of Month}}{\text{Price at Start of Month}} \right) \times 100
$$

Although we don’t have a method to convert PERMNO (a unique stock share-class level identifier) to the actual stock and calculate its price, MOM is a good enough proxy, and provides the change in price for for that month.

Using a rolling window for 2 to 64 months, we calculate additional MOM characteristics:

For $i = 1, mom_i = r_{t-1}, \quad i = 1$ where $r_{t-1}$ denotes the return in the previous month.

For $i > 1$, we calculate the momentum over a window of $i$ months by:
$$
mom_i = \left( \prod_{j=t-i}^{t-2} (r_j + 1) \right) - 1, \quad i \in \{2, \ldots, 48\},
$$
where $r_j$ denotes the return in month $j$.

These moving averages help us capture trends on time.

By using 1 to 64 window sizes, it will allow the models to capture both short-term and long-term trends. While clustering is distance and hierarchy based, we do want to add some temporal dynamics allowing the clustering algorithms to consider the timeseries behaviors.

In [4]:
if not os.path.exists(FILEPATH_PRE_PARQ):
    valid_groups = chars_df.groupby('permno').filter(lambda x: len(x) >= WINDOW and x[MOM_FEATURES[0]].isna().sum() <= 2)
    for i in tqdm(range(2, WINDOW + 1), desc="moms"):
        rolling_func = lambda x: (x + 1).rolling(window=i).apply(np.prod, raw=True) - 1
        valid_groups[f'mom{i}m'] = valid_groups.groupby('permno')[MOM_FEATURES[0]].transform(rolling_func)

    numerical_columns = valid_groups.select_dtypes(include=['float64', 'int64']).columns

    tqdm.pandas(desc="interpolate_with_median")
    valid_groups[numerical_columns] = valid_groups.groupby('permno')[numerical_columns].progress_transform(lambda x: interpolate_with_median(x,WINDOW))

    nan_percentages = valid_groups[FEATURES].isna().mean()
    valid_groups = valid_groups.drop(columns=nan_percentages[nan_percentages >= NAN_THRESHOLD].index)
    valid_groups = valid_groups.ffill().bfill()

    valid_groups.to_parquet(FILEPATH_PRE_PARQ, index=False, compression="snappy")
    chars_df = valid_groups
else:
    chars_df = pd.read_parquet(FILEPATH_PRE_PARQ)

chars_df.tail(5)

,DATE,absacc,acc,aeavol,age,agr,baspread,beta,betasq,bm,...,mom39m,mom40m,mom41m,mom42m,mom43m,mom44m,mom45m,mom46m,mom47m,mom48m
3094143,2021-12-31,0.067623,-0.067623,-0.033631,28.0,-0.364472,0.028294,1.441246,2.077190,0.048185,...,-0.160019,-0.209986,-0.207510,-0.202510,-0.141996,-0.112534,-0.138494,-0.239209,-0.193775,-0.215214
3094144,2021-12-31,0.024879,-0.024879,0.151201,37.0,0.012825,0.022583,0.958245,0.918234,0.344213,...,0.317601,0.393385,0.390042,0.374003,0.472146,0.450648,0.440951,0.408930,0.429655,0.385778
3094145,2021-12-31,0.024879,-0.024879,0.151201,37.0,0.012825,0.016800,1.255085,1.575239,0.344213,...,0.035717,0.010717,0.021282,0.021282,0.046153,0.132309,0.124301,0.012310,0.044965,0.091738
3094146,2021-12-31,0.024879,-0.024879,0.151201,37.0,0.012825,0.008384,0.866276,0.750434,0.344213,...,0.540994,0.582306,0.625281,0.623059,0.640682,0.665625,0.621044,0.595880,0.640901,0.684288
3094147,2021-12-31,0.120800,-0.120800,-0.106147,11.0,-0.519951,0.056249,1.504264,2.262812,0.033035,...,16.967672,17.179807,14.804426,18.036026,17.442083,19.366467,14.799359,14.297700,16.408472,16.549386


## Dim Reduction with PCA

Standardization and PCA at 99% variance is performed to center the data’s means for the clustering algorithms, and slightly reduce its dimensionality.

The scaler is done with the complete dataset.

In [5]:
chars_pca_df = chars_df.copy()
if os.path.exists(PCA_FILEPATH):
    pca_result_df = pd.read_pickle(PCA_FILEPATH)
else:
    scaler = StandardScaler()
    pca = PCA(MAX_VARIANCE)
    features_df = chars_pca_df.drop(['DATE', 'permno'], axis=1).bfill()
    pipeline = Pipeline([('scaler', scaler), ('pca', pca)])
    pca_df = pipeline.fit_transform(features_df)

    pca_result_df = pd.DataFrame(data=pca_df, index=chars_pca_df.index)

    pca_result_df['permno'] = chars_pca_df['permno']
    pca_result_df[MOM_FEATURES] = chars_pca_df[MOM_FEATURES]
    pca_result_df['DATE'] = chars_pca_df['DATE']

    pca_result_df.to_pickle(PCA_FILEPATH)

assert not pca_result_df.isna().any().any()

pca_result_df

,0,1,2,3,4,5,6,7,8,9,...,mom40m,mom41m,mom42m,mom43m,mom44m,mom45m,mom46m,mom47m,mom48m,DATE
0,0.265899,0.674451,-1.652120,0.395952,-0.565655,-1.103431,-0.094360,-1.419500,-0.206718,0.480775,...,0.171989,0.250122,0.240010,0.281808,0.263231,0.192639,0.639879,0.598882,0.684156,1980-01-31
1,11.967859,-3.284518,1.273922,-0.442356,0.714568,-1.284806,1.237741,0.285090,0.002585,0.428468,...,4.712689,4.822549,4.822549,4.777726,4.777726,3.454583,3.475753,2.988295,2.434365,1980-01-31
2,6.033713,0.180376,-0.333589,-0.536354,2.683301,-0.908306,-0.153683,-1.728202,-2.531858,0.244755,...,2.123810,2.276190,3.038095,3.247619,3.038095,3.152381,3.285714,4.028571,3.876190,1980-01-31
3,-0.607416,0.427278,1.818217,-0.125194,3.134476,-0.368272,1.370615,-0.577570,-1.643695,0.520253,...,0.606844,0.483240,1.031189,1.178377,0.942876,0.619064,0.324688,0.560189,0.354126,1980-01-31
4,2.632038,0.331278,-1.523178,-0.362489,1.849902,-1.739659,-0.602280,-1.495353,-0.871855,-0.145022,...,1.599627,1.517494,1.440624,1.440624,1.356836,1.435397,1.428326,1.508602,1.388189,1980-01-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3094143,-1.919868,-0.433977,-2.288112,1.879089,-1.207388,0.885580,1.322246,-0.178839,-1.260473,-0.505992,...,-0.209986,-0.207510,-0.202510,-0.141996,-0.112534,-0.138494,-0.239209,-0.193775,-0.215214,2021-12-31
3094144,-0.697372,-0.779734,-2.455765,0.805985,-0.778958,1.436156,0.571926,-0.348115,-0.680187,1.151944,...,0.393385,0.390042,0.374003,0.472146,0.450648,0.440951,0.408930,0.429655,0.385778,2021-12-31
3094145,0.689789,1.320202,-2.589043,1.458455,-1.001330,1.054757,0.394751,-0.317406,-0.663371,1.128673,...,0.010717,0.021282,0.021282,0.046153,0.132309,0.124301,0.012310,0.044965,0.091738,2021-12-31
3094146,1.413643,-0.257837,-2.872211,1.033289,-0.662100,1.233736,-0.014206,-0.258818,0.015957,0.985149,...,0.582306,0.625281,0.623059,0.640682,0.665625,0.621044,0.595880,0.640901,0.684288,2021-12-31


In [6]:
pca_result_df.columns = pca_result_df.columns.astype(str)
pca_components_cols = pca_result_df.drop('DATE', axis=1).columns
pca_components_cols

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       'mom39m', 'mom40m', 'mom41m', 'mom42m', 'mom43m', 'mom44m', 'mom45m',
       'mom46m', 'mom47m', 'mom48m'],
      dtype='object', length=139)

## KMeans

K-Means aims to minimize the sum of square distances between points (Euclidian distance) in a two-step procedure that assigns the points (xi) to a cluster (uj), and then recalculating the clusters’ centroids (uj) with the means of all the assigned data — iterated until convergence (the max clusters requested).

This is represented as:

$$
J = \sum_{i=1}^{n} \sum_{j=1}^{k} \left\| x_i^{(j)} - \mu_j \right\|^2
$$
Where:
- *n* is the total data points.
- *k* is the total clusters
- $x_{i}^{j}$ is the ith point assigned to the jth cluster
- $u_j$ is the centroid of the jth cluster.

The problem of K-Means is that it cannot identify outliers, therefore the centroids get skewed.

In this case, we will need to find the distances to the centroids, and drop anything larger than the alpha threshold of the 2 nearest neighbours — which gives the epsilon distance calculated from the helper function above.

A visualization of the algo is in the flowchart below:

![kmeans](https://raw.githubusercontent.com/adamd1985/pairs_trading_unsupervised_learning/main/images/Flowchart-of-k-means-clustering-algorithm.png)

In [7]:
if os.path.exists(KM_CLUSTERS_FILEPATH) and os.path.exists(KM_CLUSTERS_MEM_FILEPATH):
    km_models_df = pd.read_pickle(KM_CLUSTERS_FILEPATH)
    km_cluster_membership_df = pd.read_pickle(KM_CLUSTERS_MEM_FILEPATH)
else:
    # For k-means clustering, the number of clusters K = 5; 10; 50; 100; 500; 1000, and 1500 are tested
    # and K = 500 is chosen for the main results as it gives the highest Sharpe ratio.
    km_models_df, km_cluster_membership_df = train_km_clusters(pca_result_df, pca_components_cols, MOM_FEATURES, optimal_clusters=500)
    km_models_df.to_pickle(KM_CLUSTERS_FILEPATH)
    km_cluster_membership_df.to_pickle(KM_CLUSTERS_MEM_FILEPATH)

pca_result_df['km_cluster'] = km_cluster_membership_df['km_cluster']
pca_result_df[['DATE', 'km_cluster']].tail(5)

train_km_clusters:  77%|███████▋  | 387/504 [40:58<12:25,  6.37s/it]

Visualize the clusters composition:

In [ ]:
plot_cluster_distributions(km_cluster_membership_df, 'km_cluster')

## DBSCAN: Density Based Spatial Clustering or Applications with Noise

DBSCAN identifies areas of high interest in high dimensional data, which we have. It is able to handle clusters of arbitrary shapes, and can deal with outliers.

The algo groups together points that are closely packed together within ε Epsilon distance, and assigns a cluster if the number of points are greater than MinPts. Clusters are areas of high density (called core points) separated by areas of low density (called border points). Points that do not belong to any cluster (further than ε) are noise.

It then expands the radius of a cluster, adding as many neighbours as it can, or creates new clusters m core points and keeps iterating until it has visited every point.

The flowchart below gives a good visual representation of this:

![DBScan"](https://raw.githubusercontent.com/adamd1985/pairs_trading_unsupervised_learning/main/images/Flowchart-of-the-DBSCAN-algorithm.png)

We calculate the epsilon distance by finding the distances between two neighbours, order these in percentiles and finding the Alpha percentile.

In [ ]:
if os.path.exists(DBS_CLUSTERS_FILEPATH) and os.path.exists(DBS_CLUSTERS_MEM_FILEPATH):
    db_models_df = pd.read_pickle(DBS_CLUSTERS_FILEPATH)
    db_cluster_membership_df = pd.read_pickle(DBS_CLUSTERS_MEM_FILEPATH)
else:
    # For DBSCAN alpha = 0:1; : : : ; 0:9 are tested, and alpha = 0:1 is chosen for DBSCAN
    db_models_df, db_cluster_membership_df = train_db_clusters(pca_result_df, pca_components_cols, MOM_FEATURES, alpha=0.1)
    db_models_df.to_pickle(DBS_CLUSTERS_FILEPATH)
    db_cluster_membership_df.to_pickle(DBS_CLUSTERS_MEM_FILEPATH)

pca_result_df['db_cluster'] = db_cluster_membership_df['db_cluster']
pca_result_df[['DATE', 'db_cluster']].tail(5)

In [ ]:
plot_cluster_distributions(db_cluster_membership_df, 'db_cluster')

## Agglomerative
Agglomerative is a hierarchical cluster algo that works iteratively bottom-up, by merging closest pairs of clusters until it converges to one cluster, or meeting a criterion.

In our case, the criterion is the ε Epsilon distance, which means the algo will iterate until the distance between neighboring clusters is more than ε.

![Agg"](https://raw.githubusercontent.com/adamd1985/pairs_trading_unsupervised_learning/main/images/Flow-chart-of-agglomerative-hierarchical-clustering.png)

Like DBScan, epsilon is solved through the alpha parameter.

In [ ]:
if os.path.exists(AGG_CLUSTERS_FILEPATH) and os.path.exists(AGG_CLUSTERS_MEM_FILEPATH):
    agg_models_df = pd.read_pickle(AGG_CLUSTERS_FILEPATH)
    agg_cluster_membership_df = pd.read_pickle(AGG_CLUSTERS_MEM_FILEPATH)
else:
    # alpha = 0:3 for agglomerative clustering
    agg_models_df, agg_cluster_membership_df = train_agg_clusters(pca_result_df, pca_components_cols, MOM_FEATURES, alpha=0.3)
    agg_models_df.to_pickle(AGG_CLUSTERS_FILEPATH)
    agg_cluster_membership_df.to_pickle(AGG_CLUSTERS_MEM_FILEPATH)

pca_result_df['agg_cluster'] = agg_cluster_membership_df['agg_cluster']
pca_result_df[['DATE', 'agg_cluster']].tail(5)

In [ ]:
plot_cluster_distributions(agg_cluster_membership_df, 'agg_cluster')

# Trade Simulation
Our simulated trades will take the following steps:

1. Get all securities in a cluster.
1. Split into deciles based on their returns (MOM1).
1. Get returns delta of the top decile (over valued) and bottom decile (under valued)
1. Calculate the cross-sectional standard deviation of these deltas.
1. If delta > cluster’s 1 std * factor, there is a StatArb opportunity. The factor is a parameter, which in our case it’s 1.5.
1. Select securities for the Long-Short portfolio to be traded the next month.
1. Close Long-Shorts from previous month (we assume these have reversed back to the cluster’s mean).
 
All portfolios will be Equally Weighted, with their returns unaltered (w_i will be 1). We will also assume 15BPs trade fees, which will be deducted from the returns:

$$
R_p = \sum_{i=1}^{n} (w_i \cdot (r_i - 0.0015))
$$

In [ ]:
RETS_1 = MOM_FEATURES[0]
# NB: The model will trade the next time period (month, quarter, year) on the selected clusters. Paper rebalances portfolios monthly.
pca_result_df['DATE_TRADE'] = pca_result_df.groupby(['permno'])['DATE'].shift(-1).ffill()
pca_result_df[RETS] = pca_result_df.groupby(['permno'])[RETS_1].shift(-1).ffill()
trade_opportunities_km_df = trade_opportunities_db_df = trade_opportunities_agg_df = None
if not os.path.exists(SIGNALS_KM_FILEPATH):
    trade_opportunities_km_df = process_trade_opportunities(pca_result_df, 'km_cluster', SIGNALS_KM_FILEPATH)
else:
    trade_opportunities_km_df = pd.read_pickle(SIGNALS_KM_FILEPATH)

if not os.path.exists(SIGNALS_DB_FILEPATH) :
    trade_opportunities_db_df = process_trade_opportunities(pca_result_df, 'db_cluster', SIGNALS_DB_FILEPATH)
else:
    trade_opportunities_db_df = pd.read_pickle(SIGNALS_DB_FILEPATH)
if not os.path.exists(SIGNALS_AGG_FILEPATH):
    trade_opportunities_agg_df = process_trade_opportunities(pca_result_df, 'agg_cluster', SIGNALS_AGG_FILEPATH)
else:
    trade_opportunities_agg_df = pd.read_pickle(SIGNALS_AGG_FILEPATH)

In [ ]:
trade_opportunities_km_df.tail(5)

In [ ]:
trade_opportunities_db_df.tail(5)

In [ ]:
trade_opportunities_agg_df.tail(5)

## K-Means Returns

In [ ]:
overs_stock_df, unders_stock_df, overs_df, unders_df, re_unders_df, re_overs_df = get_pnl(trade_opportunities_km_df)

print(overs_df.tail(5))
print(unders_df.tail(5))
print(overs_stock_df.tail(5))
print(unders_stock_df.tail(5))

In [ ]:
plot_cummulative_rets(re_unders_df, re_overs_df)

In [ ]:
log_rets = get_log_rets(re_overs_df, re_unders_df)
log_rets['Totals'].tail(5)

In [ ]:
plot_log_rets(log_rets)

In [ ]:
km_l = get_trade_stats(unders_df[UNDER_RETS].groupby(level=0).mean())
km_o = get_trade_stats(overs_df[OVER_RETS].groupby(level=0).mean())
km_t = get_trade_stats((unders_df[UNDER_RETS] + overs_df[OVER_RETS]).groupby(level=0).mean())

km_stats_df = pd.DataFrame({
    'km_long': pd.Series(km_l),
    'km_short': pd.Series(km_o),
    'km_total': pd.Series(km_t)
})

km_stats_df

## DBScan Returns

We repeat the same for visualizations for the DBScan Algo.

In [ ]:
overs_stock_df, unders_stock_df, overs_df, unders_df, re_unders_df, re_overs_df = get_pnl(trade_opportunities_db_df)

print(overs_stock_df.groupby('DATE')[OVER_RETS].mean().tail(5))
print(unders_stock_df.groupby('DATE')[UNDER_RETS].mean().tail(5))

DATE
2021-08-31    0.056403
2021-09-30   -0.016864
2021-10-29    0.031061
2021-11-30   -0.028649
2021-12-31    0.018398
Name: over_rets, dtype: float64
DATE
2021-08-31   -0.027558
2021-09-30    0.021065
2021-10-29   -0.024961
2021-11-30    0.027246
2021-12-31   -0.029209
Name: under_rets, dtype: float64


In [ ]:
plot_cummulative_rets(re_unders_df, re_overs_df)

In [ ]:
log_rets = get_log_rets(re_overs_df, re_unders_df)
log_rets['Totals'].tail(5)

In [ ]:
plot_log_rets(log_rets)

In [ ]:
db_l = get_trade_stats(unders_df[UNDER_RETS])
db_o = get_trade_stats(overs_df[OVER_RETS])
db_t = get_trade_stats(unders_df[UNDER_RETS] + overs_df[OVER_RETS])

db_stats_df = pd.DataFrame({
    'db_long': pd.Series(db_l),
    'db_short': pd.Series(db_o),
    'db_total': pd.Series(db_t)
})

db_stats_df

## Agglomerative Returns

And for the agglomerative:

In [ ]:
overs_stock_df, unders_stock_df, overs_df, unders_df, re_unders_df, re_overs_df = get_pnl(trade_opportunities_agg_df)

print(overs_stock_df.groupby('DATE')[OVER_RETS].mean().tail(5))
print(unders_stock_df.groupby('DATE')[UNDER_RETS].mean().tail(5))

DATE
2021-08-31    0.008753
2021-09-30   -0.011504
2021-10-29    0.025349
2021-11-30   -0.028491
2021-12-31    0.015384
Name: over_rets, dtype: float64
DATE
2021-08-31   -0.008005
2021-09-30    0.019143
2021-10-29   -0.017238
2021-11-30    0.036921
2021-12-31   -0.011691
Name: under_rets, dtype: float64


In [ ]:
plot_cummulative_rets(re_unders_df, re_overs_df)

In [ ]:
log_rets = get_log_rets(re_overs_df, re_unders_df)
log_rets['Totals'].tail(5)

In [ ]:
plot_log_rets(log_rets)

In [ ]:
agg_l = get_trade_stats(unders_df[UNDER_RETS])
agg_o = get_trade_stats(overs_df[OVER_RETS])
agg_t = get_trade_stats(unders_df[UNDER_RETS] + overs_df[OVER_RETS])

agg_stats_df = pd.DataFrame({
    'agg_long': pd.Series(agg_l),
    'agg_short': pd.Series(agg_o),
    'agg_total': pd.Series(agg_t)
})

agg_stats_df

## Side-by-side Algo Comparison

Our best model was the KMeans, with an annualized 3.5% returns, max drawdown of 9%, and Sharpe of 0.01, with its long-only leg being 5 times better.

Here compared are all our models:

In [ ]:
all_stats_df = all_stats_df = pd.concat([km_stats_df, db_stats_df, agg_stats_df], axis=1)
all_stats_df

,km_long,km_short,km_total,db_long,db_short,db_total,agg_long,agg_short,agg_total
Annualized Return,0.104125,-0.069083,0.035042,0.096543,-0.068639,0.027904,0.017653,-0.011923,0.005730
Annualized Vol,0.140598,0.118432,0.050025,0.277354,0.254774,0.306651,0.146649,0.137501,0.177218
Sharpe Ratio,0.495555,-0.874212,0.011808,0.223874,-0.404635,-0.021350,-0.114542,-0.337264,-0.162065
Downside Deviation,0.110232,0.071437,0.025832,0.201862,0.198505,0.225632,0.270192,0.308342,0.348857
Sortino Ratio,0.052672,-0.120775,0.001906,0.025633,-0.043278,-0.002418,-0.005181,-0.012533,-0.006861
Max Drawdown,0.447339,0.962123,0.095789,0.963225,1.000000,1.000000,0.999957,1.000000,1.000000
Skewness,-0.538404,0.882049,2.052817,0.304520,-0.659956,0.018437,3.029819,-2.774294,0.496518
Kurtosis,4.731109,6.101469,15.509008,6.746377,10.118050,6.728692,85.107369,80.673562,59.819967


# Conclusion
In this article, we explored three clustering methods: K-Means, DBSCAN, and agglomerative clustering. Using CRSP data from Gu et al. (2020) we attempted to replicate and improve on the findings of Han et al. (2023) in their unsupervised learning for pairs trading.

In Han et al. (2023), they found that the agglomerative clustering-based strategy was the better one with an excess annualised return of 23.8% (vs ours of 3.8%), an annualised Sharpe ratio of 2.69 (vs ours of 0.4), and a maximum drawdown of 12.3% (vs ours of 20%).

In our version, K-Means was the optimal model and not the Agglomerative as in their paper. The edge that they probably had was higher quality data.